In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import xgboost as xgb
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns',100)
pd.set_option('display.width',100)
pd.set_option('display.max_colwidth',100)

tiendas_SARIMAX=['Brooklyn','Midtown_Village','Roxbury']
tiendas_XGB_24_L30 = ['Back_Bay','Yorktown']
tiendas_XGB_SC_L30 = ['Harlem','Total Tiendas']
tiendas_XGB_SC_L365 = ['Greenwich_Village','Queen_Village','South_End','Tribeca']
tiendas_XGB = tiendas_XGB_24_L30 + tiendas_XGB_SC_L30 + tiendas_XGB_SC_L365
df_hp = pd.read_csv("hiperparametros XGBOOST.csv",delimiter=";")
df_eventos = pd.read_csv("daily_calendar_with_events.csv")
df_eventos = df_eventos.dropna(subset=['event'])
df_venart = pd.read_csv("ventas por articulo.csv")
df_venart['fecha'] = pd.to_datetime(df_venart['fecha'], format = '%Y-%m-%d')

### PREDICCIONES A NIVEL DE ARTÍCULO

In [4]:
ventas_tienda_art = df_venart.groupby([df_venart['fecha'].dt.date, 'item','store'])['unidades'].sum().unstack().fillna(0).reset_index()
ventas_tienda_art.columns.name = None

predicciones_TOTALES = pd.DataFrame()
j = 0
for articulo in sorted(ventas_tienda_art['item'].unique()):
    j += 1
    print(f"art: {articulo} -> {j} de 3049 -> {j/3049}")
    ventas_art = ventas_tienda_art.loc[ventas_tienda_art['item'] == articulo]
    ventas_art = ventas_art.drop(columns=['item'])
    ventas_art['fecha'] = pd.to_datetime(ventas_art['fecha'])
    rango = pd.date_range(start="2011-01-29", end="2016-04-24")
    ventas_art = pd.merge(pd.DataFrame({'fecha': rango}), ventas_art, on='fecha', how='left')
    ventas_art = ventas_art.fillna(0)
    ventas_art.set_index('fecha', inplace=True)
    ventas_art['Total Tiendas'] = ventas_art.iloc[:, 0:].sum(axis=1)
    ventas_art['evento'] = ventas_art.index.isin(df_eventos['date']) 

    for i, tienda in enumerate(tiendas_XGB):
        print(f"{i} Tienda XGB: {tienda}")
        df = ventas_art[[tienda, 'evento']].copy()
        if tienda in tiendas_XGB_24_L30:
            df = df.loc['2014-04-24':'2016-04-24']
        # Agregar más características relevantes
        df['mes'] = df.index.month
        df['dia_semana'] = df.index.weekday
    
        # Agregar los lags de 7, 30 y 365 días
        df['lag_1'] = df[tienda].shift(1)
        df['lag_7'] = df[tienda].shift(7)
        df['lag_30'] = df[tienda].shift(30)
        if tienda in tiendas_XGB_SC_L365:
            df['lag_365'] = df[tienda].shift(365)
  
        # Eliminar filas con valores nulos que surgen por los lags
        df = df.dropna()

        # Preparar los datos para XGBoost, incluyendo las nuevas características de lag
        if tienda in tiendas_XGB_SC_L365:
            X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30', 'lag_365']].values
        
        else:
            X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30']].values
         

        y_train = df[tienda].values
     

        ne = df_hp.loc[df_hp['tienda'] == tienda, 'n_estimators'].values[0]
        md = df_hp.loc[df_hp['tienda'] == tienda, 'max_depth'].values[0]
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=ne,learning_rate=0.1, max_depth=md)
        model.fit(X_train, y_train)

        # Predecir los próximos 28 días después del conjunto de prueba
        future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
        future_eventos = pd.Series(future_dates).apply(lambda x: x in df_eventos['date'].values)
        future_mes = future_dates.month
        future_dia_semana = future_dates.weekday
    
        # Predecir los futuros 28 días (para las lags futuras se utiliza el forecast de días anteriores)
        future_lag_1 = df[tienda].shift(1).iloc[-28:].values
        future_lag_7 = df[tienda].shift(7).iloc[-28:].values
        future_lag_30 = df[tienda].shift(30).iloc[-28:].values
        if tienda in tiendas_XGB_SC_L365:
            future_lag_365 = df[tienda].shift(365).iloc[-28:].values
            X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30, future_lag_365))
        else:
            X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30))

        forecast_28_days = model.predict(X_future)

        # Crear DataFrame con las predicciones
        df_predicciones = pd.DataFrame({
            'Fecha': future_dates,
            'Prediccion_Ventas': forecast_28_days,
            'Tienda': tienda,
            'Articulo': articulo,
            'Modelo': 'XGBOOST'
      })

        # Almacenar las predicciones en la lista
        predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

    for i, tienda in enumerate(tiendas_SARIMAX):  
        print(f"{i} Tienda ARM: {tienda}")
        df = ventas_art[[tienda, 'evento']].copy()
        df['evento'] = df['evento'].astype(int)

        # ENTRENO EL MODELO CON TODOS LOS DATOS
        sarimax_model = SARIMAX(df[tienda], exog=df[['evento']],
                            order=(1, 1, 1),
                            seasonal_order=(1, 1, 1, 7),
                            initialization='approximate_diffuse')
        results_sarimax = sarimax_model.fit(disp=False)

        fechas_futuras = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
        forecast_28_days = results_sarimax.predict(start=len(df), end=len(df) + 27, exog=pd.DataFrame({'evento': [0] * 28}))

         # Crear un DataFrame para almacenar las predicciones de cada tienda
        df_predicciones = pd.DataFrame({
            'Fecha': fechas_futuras,
            'Prediccion_Ventas': forecast_28_days,
            'Tienda': tienda,
            'Articulo': articulo,
            'Modelo': 'SARIMAX'
            })

        # Almacenar las predicciones en la lista
        predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

predicciones_TOTALES.to_csv('Pred_uds_art.txt')

art: ACCESORIES_1_001 -> 1 de 3049 -> 0.00032797638570022957
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury
art: ACCESORIES_1_002 -> 2 de 3049 -> 0.0006559527714004591
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury
art: ACCESORIES_1_003 -> 3 de 3049 -> 0.0009839291571006887
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM:

In [19]:
df_venart

,fecha,ejercicio,mes,semana,store_code,category,department,item,unidades,cifra,region,store,latitud,longitud
0,2011-03-09,2011,3,10,BOS_1,ACCESORIES,ACCESORIES_1,ACCESORIES_1_002,1.00,5.28,Boston,South_End,42.34,-71.07
1,2011-03-11,2011,3,10,BOS_1,ACCESORIES,ACCESORIES_1,ACCESORIES_1_002,1.00,5.28,Boston,South_End,42.34,-71.07
2,2011-03-12,2011,3,10,BOS_1,ACCESORIES,ACCESORIES_1,ACCESORIES_1_002,1.00,5.28,Boston,South_End,42.34,-71.07
3,2011-03-13,2011,3,10,BOS_1,ACCESORIES,ACCESORIES_1,ACCESORIES_1_002,1.00,5.28,Boston,South_End,42.34,-71.07
4,2011-03-19,2011,3,11,BOS_1,ACCESORIES,ACCESORIES_1,ACCESORIES_1_002,1.00,5.28,Boston,South_End,42.34,-71.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18550271,2016-04-03,2016,4,13,PHI_3,SUPERMARKET,SUPERMARKET_3,SUPERMARKET_3_827,1.00,1.20,Philadelphia,Queen_Village,39.94,-75.15
18550272,2016-04-04,2016,4,14,PHI_3,SUPERMARKET,SUPERMARKET_3,SUPERMARKET_3_827,2.00,2.40,Philadelphia,Queen_Village,39.94,-75.15
18550273,2016-04-05,2016,4,14,PHI_3,SUPERMARKET,SUPERMARKET_3,SUPERMARKET_3_827,3.00,3.60,Philadelphia,Queen_Village,39.94,-75.15
18550274,2016-04-06,2016,4,14,PHI_3,SUPERMARKET,SUPERMARKET_3,SUPERMARKET_3_827,2.00,2.40,Philadelphia,Queen_Village,39.94,-75.15


In [21]:
ventas_tienda = df_venart.groupby([df_venart['fecha'].dt.date,'store'])['unidades'].sum().unstack().fillna(0).reset_index()
ventas_tienda.columns.name = None
rango = pd.date_range(start="2011-01-29", end="2016-04-24")
ventas_tienda['fecha'] = pd.to_datetime(ventas_tienda['fecha'], format = '%Y-%m-%d')
ventas_tienda = pd.merge(pd.DataFrame({'fecha': rango}), ventas_tienda, on='fecha', how='left')
ventas_tienda = ventas_tienda.fillna(0)
ventas_tienda.set_index('fecha', inplace=True)
ventas_tienda['Total Tiendas'] = ventas_tienda.iloc[:, 0:].sum(axis=1)
ventas_tienda['evento'] = ventas_tienda.index.isin(df_eventos['date']) 

In [22]:
ventas_tienda

,Back_Bay,Brooklyn,Greenwich_Village,Harlem,Midtown_Village,Queen_Village,Roxbury,South_End,Tribeca,Yorktown,Total Tiendas,evento
fecha,,,,,,,,,,,,
2011-01-29,3030.00,1625.00,4337.00,3494.00,2704.00,4038.00,3852.00,2556.00,4739.00,2256.00,32631.00,False
2011-01-30,3006.00,1777.00,4155.00,3046.00,2194.00,4198.00,3937.00,2687.00,4827.00,1922.00,31749.00,False
2011-01-31,2225.00,1386.00,2816.00,2121.00,1562.00,3317.00,2731.00,1822.00,3785.00,2018.00,23783.00,False
2011-02-01,2169.00,1440.00,3051.00,2324.00,1251.00,3211.00,2954.00,2258.00,4232.00,2522.00,25412.00,False
2011-02-02,1726.00,1536.00,2630.00,1942.00,2.00,2132.00,2492.00,1694.00,3817.00,1175.00,19146.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-20,3315.00,2500.00,3722.00,3691.00,3242.00,3159.00,3384.00,2901.00,5235.00,4194.00,35343.00,False
2016-04-21,3380.00,2458.00,3709.00,3303.00,3324.00,3226.00,3446.00,2776.00,5018.00,4393.00,35033.00,False
2016-04-22,3691.00,2628.00,4387.00,4457.00,3991.00,3828.00,3902.00,3022.00,5623.00,4988.00,40517.00,False


In [34]:
articulo_consulta = 'SUPERMARKET_3_090'
tienda_consulta = 'Tribeca'
ventas_articulo = df_venart[(df_venart['item'] == articulo_consulta) & (df_venart['store'] == tienda_consulta)]
ventas_articulo = ventas_articulo.groupby([ventas_articulo['fecha'].dt.date,'store'])['unidades'].sum().unstack().fillna(0).reset_index()
ventas_articulo.columns.name = None
rango = pd.date_range(start="2011-01-29", end="2016-04-24")
ventas_articulo['fecha'] = pd.to_datetime(ventas_articulo['fecha'], format = '%Y-%m-%d')
ventas_articulo = pd.merge(pd.DataFrame({'fecha': rango}), ventas_articulo, on='fecha', how='left')
ventas_articulo = ventas_articulo.fillna(0)
ventas_articulo.set_index('fecha', inplace=True)


predicciones_articulo = predicciones_TOTALES[(predicciones_TOTALES['Articulo'] == articulo_consulta) & (predicciones_TOTALES['Tienda'] == tienda_consulta)]
predicciones_articulo = predicciones_articulo.set_index('Fecha')
print (ventas_articulo)
print (predicciones_articulo)

# VISUALIZACIÓN DE PREDICCIONES
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=ventas_articulo.index, y=ventas_articulo[tienda_consulta], mode='lines', name='Datos históricos'))
fig.add_trace(go.Scatter(x=predicciones_articulo.index, y=predicciones_articulo['Prediccion_Ventas'], mode='lines', name='Predicciones'))      
    
# AÑADO LAS MARCAS DE LOS DÍAS FESTIVOS POR SI AYUDAN A INTERPORETAR EL MODELO
for _, evento in df_eventos.iterrows():
    evento_date = evento['date']
    evento_name = evento['event']
    fig.add_vline(x=evento_date, line=dict(color="red", width=2, dash="dash"))
    fig.add_annotation(x=evento_date, y=1, text=evento_name, showarrow=True, arrowhead=1, textangle=-90, valign="top")
  
    #fig.add_annotation(x=evento_date, y=max(ventas_articulo[tienda_consulta].max(), max(predicciones_articulo['Prediccion_Ventas'])), text=evento_name, showarrow=True, arrowhead=1, textangle=-90, valign="top")
  
    # ALGUNOS AJUSTES DE DISEÑO DE LOS GRÁFICOS
fig.update_layout(
    title= f'{tienda_consulta} Predicción Final Artículo {articulo_consulta}',
    xaxis_title='Fecha',
    yaxis_title=tienda_consulta,
    legend=dict(x=0.4, y=1.4),
    xaxis=dict(
        tickformat='%Y-%m-%d',
        tickmode='auto',
        nticks=100,  # Ajusta el número de etiquetas en el eje x
        range=['2015-05-24', '2016-05-23']  # Limita el rango de fechas
    ),
    template='plotly_white'
)

fig.show()

            Tribeca
fecha              
2011-01-29   108.00
2011-01-30   132.00
2011-01-31   102.00
2011-02-01   120.00
2011-02-02   106.00
...             ...
2016-04-20    88.00
2016-04-21    77.00
2016-04-22   141.00
2016-04-23   139.00
2016-04-24   130.00

[1913 rows x 1 columns]
            Prediccion_Ventas   Tienda           Articulo   Modelo
Fecha                                                             
2016-04-25              97.88  Tribeca  SUPERMARKET_3_090  XGBOOST
2016-04-26              73.95  Tribeca  SUPERMARKET_3_090  XGBOOST
2016-04-27              81.69  Tribeca  SUPERMARKET_3_090  XGBOOST
2016-04-28             118.32  Tribeca  SUPERMARKET_3_090  XGBOOST
2016-04-29             168.73  Tribeca  SUPERMARKET_3_090  XGBOOST
2016-04-30             179.60  Tribeca  SUPERMARKET_3_090  XGBOOST
2016-05-01             123.77  Tribeca  SUPERMARKET_3_090  XGBOOST
2016-05-02              97.33  Tribeca  SUPERMARKET_3_090  XGBOOST
2016-05-03              86.46  Tribeca  SUPER

In [37]:
import pandas as pd
from datetime import timedelta

articulo_consulta = 'SUPERMARKET_3_090'
tienda_consulta = 'Tribeca'
ventas_articulo = df_venart[(df_venart['item'] == articulo_consulta) & (df_venart['store'] == tienda_consulta)]
ventas_articulo = ventas_articulo.groupby([ventas_articulo['fecha'].dt.date,'store'])['unidades'].sum().unstack().fillna(0).reset_index()
ventas_articulo.columns.name = None
rango = pd.date_range(start="2011-01-29", end="2016-04-24")
ventas_articulo['fecha'] = pd.to_datetime(ventas_articulo['fecha'], format = '%Y-%m-%d')
ventas_articulo = pd.merge(pd.DataFrame({'fecha': rango}), ventas_articulo, on='fecha', how='left')
ventas_articulo = ventas_articulo.fillna(0)
ventas_articulo.set_index('fecha', inplace=True)
predicciones_articulo = predicciones_TOTALES[(predicciones_TOTALES['Articulo'] == articulo_consulta) & (predicciones_TOTALES['Tienda'] == tienda_consulta)]
predicciones_articulo = predicciones_articulo.set_index('Fecha')

# Obtén la fecha máxima y calcula el rango de los últimos 365 días
fecha_maxima = predicciones_articulo.index.max()  # Última fecha de predicción
fecha_maxima = ventas_articulo.index.max() + timedelta(days=7) # Última fecha de predicción
fecha_minima = fecha_maxima - timedelta(days=365)  # 365 días atrás

# Filtra los datos de ventas y predicciones para los últimos 365 días
ventas_filtradas = ventas_articulo[tienda_consulta].loc[fecha_minima:fecha_maxima]
predicciones_filtradas = predicciones_articulo['Prediccion_Ventas'].loc[fecha_minima:fecha_maxima]
max_valor = max(ventas_filtradas.max(), predicciones_filtradas.max())

fig = go.Figure()
fig.add_trace(go.Scatter(x=ventas_filtradas.index, y=ventas_filtradas, mode='lines', name='Datos históricos'))
fig.add_trace(go.Scatter(x=predicciones_filtradas.index, y=predicciones_filtradas, mode='lines', name='Predicciones'))

# Añadir eventos
for _, evento in df_eventos.iterrows():
    evento_date = evento['date']
    evento_name = evento['event']
    fig.add_vline(x=evento_date, line=dict(color="red", width=2, dash="dash"))
    fig.add_annotation(x=evento_date, y=max_valor, text=evento_name, showarrow=True, arrowhead=1, textangle=-90, valign="top")

# Ajustes del diseño, incluyendo el límite del eje y
fig.update_layout(
    title=f'{tienda_consulta} Predicción Final Artículo {articulo_consulta}',
    xaxis_title='Fecha',
    yaxis_title=tienda_consulta,
    legend=dict(x=0.4, y=1.4),
    xaxis=dict(
        tickformat='%Y-%m-%d',
        tickmode='auto',
        nticks=100,  # Ajusta el número de etiquetas en el eje x
        range=[str(fecha_minima), str(fecha_maxima)]  # Limita el rango de fechas a los últimos 365 días
    ),
    yaxis=dict(
        range=[0, max_valor + 0.1 * max_valor]  # Aumenta un 10% para tener margen superior
    ),
    template='plotly_white'
)
fig.show()
